In [1]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

In [2]:
import numpy as np
import pandas as pd
from PIL import Image, ImageOps

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
train_data = pd.read_csv('/kaggle/input/hackathon23/new_data_lun.csv')
# для тренування на всіх даних видаліть + random split на train_dataframe і val_dataframe (далі комірка)
# train_data = train_data[:20000]

In [4]:
train_dataframe, val_dataframe = train_test_split(train_data, train_size=0.8)

In [5]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, batch_size=64, image_size=(224, 224)):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        batch_df = self.dataframe[index * self.batch_size:(index + 1) * self.batch_size]

        batch_images1 = []
        batch_images2 = []
        batch_labels = []

        for _, row in batch_df.iterrows():
            img1_url, img2_url, label = row['image_url1'], row['image_url2'], row['is_same']
            image1 = Image.open(os.path.join('/kaggle/input/hackathon23/train-files/images/', img1_url))
            image2 = Image.open(os.path.join('/kaggle/input/hackathon23/train-files/images/', img2_url))
            if image1.mode == "L" or image1.mode == "RGBA":
                image1 = image1.convert("RGB")
            if image2.mode == "L" or image2.mode == "RGBA":
                image2 = image2.convert("RGB")
            
            image1 = image1.resize((224, 224))  
            image2 = image2.resize((224, 224))  
                
            image1 = tf.convert_to_tensor(image1)
            image2 = tf.convert_to_tensor(image2)

            batch_images1.append(image1)
            batch_images2.append(image2)
            batch_labels.append(label)

        batch_images1 = np.array(batch_images1) / 255.0  # Normalize images
        batch_images2 = np.array(batch_images2) / 255.0  # Normalize images
        batch_labels = np.array(batch_labels)
        return batch_images1, batch_images2, batch_labels

    def on_epoch_end(self):
        self.dataframe = self.dataframe.sample(frac=1)

In [6]:
train_data_generator = DataGenerator(train_dataframe, batch_size=32)
val_data_generator = DataGenerator(val_dataframe, batch_size=32)

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_data_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.int32)
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: val_data_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.int32)
    )
)

In [7]:
def cosine_similarity_batched(x1, x2):
    # Compute L2-norm of each vector
    x1_norm = tf.linalg.norm(x1, axis=1, keepdims=True)
    x2_norm = tf.linalg.norm(x2, axis=1, keepdims=True)

    # Compute dot product between vectors
    dot_product = tf.reduce_sum(x1 * x2, axis=1, keepdims=True)

    # Compute cosine similarity
    cosine_similarity = dot_product / (x1_norm * x2_norm + tf.keras.backend.epsilon())

    return cosine_similarity

In [8]:
def custom_f1_score(y_true, y_pred):
    # Calculate true positives, false positives, and false negatives
    TP = tf.math.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred, 1)), dtype=tf.float32), axis=0)
    FP = tf.math.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(y_pred, 1)), dtype=tf.float32), axis=0)
    FN = tf.math.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred, 0)), dtype=tf.float32), axis=0)

    # Calculate precision, recall, and F1 score
    precision = TP / (TP + FP + tf.keras.backend.epsilon())
    recall = TP / (TP + FN + tf.keras.backend.epsilon())
    f1_score = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return f1_score

In [9]:
from tensorflow.keras import Input, Sequential, Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import applications, layers, losses, optimizers, metrics

mobile_net = VGG16(
    weights="imagenet", input_shape=(224,224, 3), include_top=False
)

flatten = layers.Flatten()(mobile_net.output)
x1 = layers.Dense(512, activation="relu")(flatten)
x1 = layers.BatchNormalization()(x1)
x2 = layers.Dense(256, activation="relu")(x1)
x2 = layers.BatchNormalization()(x2)
output = layers.Dense(256)(x2)

embedding = Model(mobile_net.input, output, name="Embedding")

trainable = False
for layer in mobile_net.layers:
    if layer.name == "block5_conv1":
        trainable = True
    layer.trainable = trainable

58889256/58889256 [==============================] - 3s 0us/step


In [10]:
embedding.summary()

Model: "Embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0 

In [11]:
import tensorflow as tf
from tensorflow.keras import backend as K


def create_siamese_model(input_shape):
    # Define inputs
    input_1 = layers.Input(name="input_1", shape=input_shape)
    input_2 = layers.Input(name="input_2", shape=input_shape)

    # Get embeddings from base models
    embedding_1 = embedding(input_1)
    embedding_2 = embedding(input_2)

    # Compute cosine similarity
    cosine_similarity = cosine_similarity_batched(embedding_1, embedding_2)

    # Create siamese model
    siamese_model = tf.keras.Model(inputs=[input_1, input_2], outputs=cosine_similarity)
    return siamese_model

In [12]:
# Custom training method
def train_model(model, train_dataset, val_dataset, threshold):
    # Define loss function
    loss_fn = tf.keras.losses.BinaryCrossentropy()

    # Define metrics
    train_loss = tf.keras.metrics.Mean()
    train_f1 = tf.keras.metrics.Mean()
    
    best_metric = np.inf

    val_loss = tf.keras.metrics.Mean()
    val_f1 = tf.keras.metrics.Mean()

    # Define optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

    @tf.function
    def train_step(x1, x2, labels):
        with tf.GradientTape() as tape:
            # Forward pass
            predictions = model([x1, x2])
            labels = tf.expand_dims(labels, axis=1)
            loss_value = loss_fn(labels, predictions)

        # Backward pass
        gradients = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # Compute F1 score
        f1_value = custom_f1_score(labels, tf.cast(predictions > threshold, tf.int32))

        return loss_value, f1_value

    @tf.function
    def val_step(x1, x2, labels):
        # Forward pass
        predictions = model([x1, x2])
        labels = tf.expand_dims(labels, axis=1)
        loss_value = loss_fn(labels, predictions)

        # Compute F1 score
        f1_value = custom_f1_score(labels, tf.cast(predictions > threshold, tf.int32))

        return loss_value, f1_value

    # Training loop
    epochs = 10
    for epoch in range(epochs):
        # Reset metrics
        train_loss.reset_states()
        train_f1.reset_states()
        val_loss.reset_states()
        val_f1.reset_states()

        # Training
        for x1, x2, labels in train_dataset:
            loss_value, f1_value = train_step(x1, x2, labels)
            train_loss.update_state(loss_value)
            train_f1.update_state(f1_value)

        # Validation
        for x1, x2, labels in val_dataset:
            loss_value, f1_value = val_step(x1, x2, labels)
            val_loss.update_state(loss_value)
            val_f1.update_state(f1_value)

        if val_loss.result() < best_metric:
            best_metric = val_loss.result()
            model.save('/kaggle/working/best_model.h5')
        # Print progress
        print(f"Epoch {epoch+1}/{epochs}:")
        print(f"Train Loss: {train_loss.result()}, Train F1: {train_f1.result()}")
        print(f"Val Loss: {val_loss.result()}, Val F1: {val_f1.result()}")
        print()

# Example usage
input_shape = (224, 224, 3)
threshold = 0.75

# Create the model
model = create_siamese_model(input_shape)

# Train the model
train_model(model, train_dataset, val_dataset, threshold)

Epoch 1/10:
Train Loss: 0.07818122953176498, Train F1: 0.9778134822845459
Val Loss: 0.058450888842344284, Val F1: 0.9772791266441345

Epoch 2/10:
Train Loss: 0.06491358578205109, Train F1: 0.978593647480011
Val Loss: 0.05733633413910866, Val F1: 0.9763917922973633

Epoch 3/10:
Train Loss: 0.06479302793741226, Train F1: 0.9786688089370728
Val Loss: 0.07663700729608536, Val F1: 0.9725774526596069

Epoch 4/10:
Train Loss: 0.06383698433637619, Train F1: 0.9783222079277039
Val Loss: 0.058385759592056274, Val F1: 0.9792412519454956

Epoch 5/10:
Train Loss: 0.04369734600186348, Train F1: 0.9834318161010742
Val Loss: 0.05075804889202118, Val F1: 0.982099711894989

Epoch 6/10:
Train Loss: 0.040756333619356155, Train F1: 0.9850804805755615
Val Loss: 0.05211348831653595, Val F1: 0.9828423261642456

Epoch 7/10:
Train Loss: 0.030423294752836227, Train F1: 0.9874131679534912
Val Loss: 0.04625372216105461, Val F1: 0.9835363030433655

Epoch 8/10:
Train Loss: 0.02603120729327202, Train F1: 0.9896273612